# Optimize Your Recipe

In this tutorial we will optimize the model exported in the last secion, the Application Framework [Getting Started](/af/latest/notebooks/GettingStarted/).

## Set Up

If you have not done so already, please ensure your environment is set up according to [Setting up your LEIP Environment](/home/content/tutorials/environment/). This will ensure that while executing this notebook the APIs of both the [Application Framework](/af/latest/content/) and [Compiler Framework](/cf/latest/content/) are accessible.

First we import the necessary object definitions and initialize the connection to the leip-server.
The code in the following cell assumes your license key was set in the `LICENSE_KEY` environment variable in the environment the notebook is run from. Alternatively it can be pasted as a string into the last parameter of the `leip.create_profile()` call.

In [1]:
from leip_client import (
    Leip,
    EvaluateTask,
    TaskModelOptions,
    DatasetOptions,
    PascalVOCOptions,
    COCOOptions,
    CompressOptions,
    CompileOptions,
    PipelineTask,
    PipelineInnerTask,
    CompilePipelineOptions,
    OptimizePipelineOptions,
)
from commons.serialization import pretty_print
from pathlib import Path
import json
import os
import time

# initialize leip_client
leip = Leip.load_instance()
leip.create_profile("default", "http://host.docker.internal:8888", os.environ["LICENSE_KEY"])
leip.check_connection(silent=False)

Checking connection... ✅


We will set a few variable that will be used below.

The `model_folder` and `dataset_folder` should be set to match the folders where the model and dataset were saved in the last tutorial, the Application Framework [Getting Started](/af/latest/notebooks/GettingStarted/).

In [2]:
model_folder = "/recipe_test/exported_model"  # TODO FIXME
dataset_folder = "/workspace/datasets/road-sign-data"  # TODO FIXME
target = "cuda"
target_host = "llvm -mcpu=skylake"

## Optimize

Next we will use the Pipeline API to group together a couple of subtasks into one batched execution. One subtask will optimize the model into `int8` for its target, and the other subtask will compile to `float32`.

The optimize step requires a *representative dataset* which is used for calibration during the quantization of activations in the model. A good rule of thumb for creating a representative dataset is to include at least one image from each output class in the dataset. Often though, even a few input samples from the dataset are enough to get good quantized accuracy for the model. Other times the success of the quantization can be very sensitive to the representative dataset chosen. So you might want to experiment in this area. We will chose a simple representative dataset of 10 randomly selected images for this tutorial.


In [3]:
# make rep_dataset
rep_dataset_items = [
    "/recipe_test/MSCocodata/validation/data/000000000139.jpg",
    "/recipe_test/MSCocodata/validation/data/000000000285.jpg",
    "/recipe_test/MSCocodata/validation/data/000000000632.jpg",
    "/recipe_test/MSCocodata/validation/data/000000000724.jpg",
    "/recipe_test/MSCocodata/validation/data/000000000776.jpg",
    "/recipe_test/MSCocodata/validation/data/000000000785.jpg",
    "/recipe_test/MSCocodata/validation/data/000000000802.jpg",
    "/recipe_test/MSCocodata/validation/data/000000000872.jpg",
    "/recipe_test/MSCocodata/validation/data/000000000885.jpg",
    "/recipe_test/MSCocodata/validation/data/000000001000.jpg",
]
rep_dataset_file = Path("/recipe_test/rep_dataset.txt")
rep_dataset_file.write_text("\n".join(rep_dataset_items))

569

Next we will run LEIP Pipeline. The `model_options`, `compress_options` and `compile_options` together comprise the various options needed to optimize and compile the model. Refer to the [Compiler Framework documentation](/cf/latest/content/) for more information on the available options. 

In [5]:
# run pipeline
model_options = TaskModelOptions(
    path="${vars.model_path}",
    task_family="detection",
)

compress_options = CompressOptions(
    rep_dataset="/recipe_test/rep_dataset.txt",
    quantizer="symmetricpc",  # "symmetric", "symmetricpc"
    # calibration_method="average",  # "minmax"
    # optimization=["tensor_splitting"],
)

compile_options = compile=CompileOptions(
    target=target,
    target_host=target_host,
)

pipeline_config = PipelineTask(
    vars={
        "model_path": str(model_folder)
    },
    name="RecipePipeline",
    tasks=[
        PipelineInnerTask(
            name="Int8",
            optimize=OptimizePipelineOptions(
                model=model_options,
                compress=compress_options,
                compile=compile_options,
            ),
        ),
        PipelineInnerTask(
            name="Float32",
            compile=CompilePipelineOptions(
                model=model_options,
                target=compile_options.target,
                target_host=compile_options.target_host,
            ),
        ),
    ],
)
results = pipeline_config.run()
pretty_print(results)


id        : 9
name      : PipelineTask
status    : completed
options   :
    name  : RecipePipeline
    vars  :
        model_path : /recipe_test/exported_model
    tasks :
        -
            name     : Int8
            optimize :
                model    :
                    task_family : detection
                    path        : /recipe_test/exported_model
                compress :
                    quantizer   : symmetricpc
                    rep_dataset : /recipe_test/rep_dataset.txt
                compile  :
                    target       : cuda -keys=cuda,gpu -arch=sm_86 -max_num_threads=1024 -thread_warp_size=32
                    target_host  : llvm -mcpu=skylake
                    use_tensorrt : Yes
        -
            name    : Float32
            compile :
                model        :
                    task_family : detection
                    path        : /recipe_test/exported_model
                target       : cuda -keys=cuda,gpu -arch=sm_86 -max

## Evaluate

Now we will use the Evaluate API to run an evaluation of the optimized model on the server. This will return a mean average precision (mAP) for how well the optimized model does on the validation set of the dataset in our detection task.

 Refer to [the Evaluate module](/cf/3.0/content/modules/evaluate/) for more information on using LEIP Evaluate for evaluating the acurracy of a model throughout the various stages in your pipeline.


In [ ]:
# evaluate int8

dataset_options = DatasetOptions(
    root_dir=dataset_folder,
    voc=PascalVOCOptions(
        data_dir=Path("images"),
        # label_map_file=Path("JPEGImages"),  # TODO FIXME
        annotations_dir=Path("annotations"),
    ),
)

evaluate_task = EvaluateTask(
    model=TaskModelOptions(
        path=results.results["tasks"][0]["value"]["output_path"],
        task_family="detection",
    ),
    dataset=dataset_options,
    # test_size=5000,
)

model_eval_job = evaluate_task.run()
pretty_print(model_eval_job)
mAP = model_eval_job.results["scoring"]["mAP"]["0.5:0.95:0.05"]
print(f"int8 mAP: {mAP}")

## Next Steps

We have optimized our model using LEIP Optimize and evaluated how well the optimized model performs using LEIP Evaluate. We will continue with the next tutorial where we deploy the model using the [Runtime Framework](/rf/content/).